# Creating and filling a consolidated table of M-ligue results

### 1. Setup

Define base folders, filenames, and URLs.
Only change the tournament_name and url each time you process a new tournament.

In [ ]:
from pathlib import Path
import pandas as pd
import urllib.request
from openpyxl import load_workbook

# Base folder where game results are stored
base_dir = Path(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Team\Games results")

# Consolidated results file
consolidated_path = base_dir / "Consolidated.xlsx"

# Define current tournament
tournament_name = "2025 July Final"

# Path for the downloaded tournament file
destination = base_dir / f"{tournament_name}.xlsx"

# Download URL for the tournament results
url = "https://chgk-world.ru/tournaments/1166/table/simple/verbose.xlsx"

### 2. Create Consolidated table for all tournaments

Run this step only the first time you set up the consolidated file.
It creates an empty Excel file with all required columns.

In [ ]:
if not consolidated_path.exists():
    columns = ['Rank', 'Team', 'City', 'ID', 'Captain', 'Location', 'Status',
               'Total', 'Time', 'Question', 'Result', 'Tournament']
    pd.DataFrame(columns=columns).to_excel(consolidated_path, index=False)
    print("Created new Consolidated.xlsx file.")
else:
    print("Consolidated.xlsx already exists, skipping creation.")

### 3. Prepare Consolidated table

Start here if the file already exists.

In [ ]:
df_consolidated = pd.read_excel(consolidated_path)
print(f"Loaded Consolidated file with {len(df_consolidated)} rows.")

### 4. Download the results of the particular game

After a game finishes, you can download its Excel results directly from chgk-world.ru.

In [ ]:
urllib.request.urlretrieve(url, destination)
print(f"Downloaded results to {destination}")

### 5. Change '+' and '-' to '1' and '0'

In the downloaded results, correct and incorrect answers are shown as + and –.
We’ll convert them to numeric values (1 for correct, 0 for incorrect) for easier processing later.

In [ ]:
replacement_pair = {"+": 1, "-": 0, "?": 0, None: 0, "": 0}

wb = load_workbook(destination)
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if cell.value in replacement_pair:
                cell.value = replacement_pair[cell.value]
wb.save(destination)

print("Replaced symbols with numeric values.")

### 6. Reshaping the table

Each question’s result is stored in a separate column.
We’ll convert those columns into rows so that each question gets its own record.

In [ ]:
df_game = pd.read_excel(destination)

id_vars = [
    "#", "Название", "Город", "Makid", "Капитан", 
    "Площадка", "Зачет", "Сумма", "Время+"
]

df_game = df_game.melt(id_vars=id_vars, var_name="Question", value_name="Result")
df_game.to_excel(destination, index=False)
print("Transformed questions from columns to rows.")

### 7. Rename columns

Rename Russian headers to English ones for consistency in later visualization.

In [ ]:
df_game.rename(columns={
    "#": "Rank",
    "Название": "Team",
    "Город": "City",
    "Makid": "ID",
    "Капитан": "Captain",
    "Площадка": "Location",
    "Зачет": "Status",
    "Сумма": "Total",
    "Время+": "Time"
}, inplace=True)

print("Renamed columns.")

### 8. Add Tournament Name

Add a column to keep track of which tournament each record belongs to.

In [ ]:
df_game["Tournament"] = tournament_name
df_game.to_excel(destination, index=False)

print(f"Added tournament name: {tournament_name}")

### 9. Append Tournament Results to the Consolidated File

Finally, merge the cleaned tournament data into your consolidated table and save it.

In [ ]:
df_updated = pd.concat([df_consolidated, df_game], ignore_index=True)
df_updated.to_excel(consolidated_path, index=False)

print(f"Appended {len(df_game)} new rows to Consolidated.xlsx.")